In [1]:
import os


In [2]:
# Function that create the required folders and defines paths for the data processing 
def init_paths(analysis_name : str,
               database_name : str) -> list:
    
    """
    Function that create the required folders and defines paths for the data processing.
    the folders final path will be: `main_folder/analysis_name/database_name`, for example.
    the output will be list of 4 paths:"data_raw/...", "data_temp/...", "results_figures/...", "results_tables/..."
    analysis name : str -> name of the anlysis performed, for example "lpa_analysis".
    database_name : name of the database, imported from the `config.json` file.
    """
    
    # Defining and creating the main folders
    paths = ["data_raw", "data_temp", "results_figures", "results_tables"]
    create_folders(paths)

    # Defining and creating the sub-folders according to the input arguments.
    for i in [analysis_name, database_name]:
        paths = [os.path.join(j, i) for j in paths]
        create_folders(paths)

    return paths

In [ ]:
from source.helpers import read_json, create_folders, mysql_qry
from tqdm import tqdm


class sub_mut():
    def __init__(self):
        
        # Getting the config information and database name
        self.config_info = read_json("config.json")
        self.db_name = self.config_info["database"]["db_name"]

        # Initilizing the analysis paths and folders
        # folder output order: "data_raw", "data_temp", "results_figures", "results_tables"
        self.analysis_type = "substitution_sruvival"
        self.paths = init_paths(self.analysis_type, self.db_name)

        # Importing the required raw tables from the MySQL server
        req_folders = self.config_info[self.analysis_type]["req_tables"].split(",")
        sql_engine = mysql_qry()
        
        for i in tqdm(req_folders, desc="Importing MySQL tables", unit="tables"):
            temp_qry = f"""
                        SELECT * FROM {self.db_name}.{i}
                        """
            temp_df = sql_engine.run_qry(temp_qry)
            temp_df.to_csv(os.path.join(self.paths[0],f"{i}.csv"))
        
        sql_engine.close_conn()

        # > Need to verify the existance of tables before import
        # > 



sub_mut()

> folder `data_raw` exists, continuing.
> folder `data_temp` exists, continuing.
> folder `results_figures` exists, continuing.
> folder `results_tables` exists, continuing.
> folder `data_raw\substitution_sruvival` exists, continuing.
> folder `data_temp\substitution_sruvival` exists, continuing.
> folder `results_figures\substitution_sruvival` exists, continuing.
> folder `results_tables\substitution_sruvival` exists, continuing.
> folder `data_raw\substitution_sruvival\covid_vaccine_new` exists, continuing.
> folder `data_temp\substitution_sruvival\covid_vaccine_new` exists, continuing.
> folder `results_figures\substitution_sruvival\covid_vaccine_new` exists, continuing.
> folder `results_tables\substitution_sruvival\covid_vaccine_new` exists, continuing.
> Established connecntion to the covid_vaccine_new database.


Importing MySQL tables: 100%|██████████| 1/1 [00:01<00:00,  1.69s/tables]

> Query executed successfully: 

                        SELECT * FROM covid_vaccine_new.sample_metadata
                        
> MySQL connenction terminated.
